<a href="https://colab.research.google.com/github/vbloise3/Data_Science/blob/master/HypothesisandInference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from typing import Tuple 
import math
import random

def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p: float,
                       mu: float = 0,
                       sigma: float = 1,
                       tolerance: float = 0.00001) -> float:
    """Find approximate inverse using binary search"""

    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)

    low_z = -10.0                      # normal_cdf(-10) is (very close to) 0
    hi_z  =  10.0                      # normal_cdf(10)  is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # Consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            low_z = mid_z              # Midpoint too low, search above it
        else:
            hi_z = mid_z               # Midpoint too high, search below it

    return mid_z

def normal_approximation_to_binomial( n: int, p: float)-> Tuple[ float, float]: 
  """ Returns mu and sigma corresponding to a Binomial( n, p)""" 
  mu = p * n 
  sigma = math.sqrt( p * (1-p) * n) 
  return mu, sigma

# The normal cdf _is_ the probability the variable is below a threshold 
normal_probability_below = normal_cdf

# It's above the threshold if it's not below the threshold 
def normal_probability_above( lo: float, mu: float = 0, sigma: float = 1)-> float: 
  """ The probability that an N( mu, sigma) is greater than lo.""" 
  return 1-normal_cdf( lo, mu, sigma)

# It's between if it's less than hi, but not less than lo 
def normal_probability_between( lo: float, hi: float, mu: float = 0, sigma: float = 1)-> float: 
  """ The probability that an N( mu, sigma) is between lo and hi.""" 
  return normal_cdf( hi, mu, sigma)-normal_cdf( lo, mu, sigma)

# It's outside if it's not between 
def normal_probability_outside( lo: float, hi: float, mu: float = 0, sigma: float = 1)-> float: 
  """ The probability that an N( mu, sigma) is not between lo and hi.""" 
  return 1-normal_probability_between( lo, hi, mu, sigma)

def normal_upper_bound( probability: float, mu: float = 0, sigma: float = 1)-> float: 
  """ Returns the z for which P( Z < = z) = probability""" 
  return inverse_normal_cdf( probability, mu, sigma)

def normal_lower_bound( probability: float, mu: float = 0, sigma: float = 1)-> float: 
  """ Returns the z for which P( Z > = z) = probability""" 
  return inverse_normal_cdf( 1-probability, mu, sigma)

def normal_two_sided_bounds( probability: float, mu: float = 0, sigma: float = 1)-> Tuple[ float, float]: 
  """ Returns the symmetric (about the mean) bounds that contain the specified probability """ 
  tail_probability = (1-probability) / 2 
  # upper bound should have tail_probability above it 
  upper_bound = normal_lower_bound( tail_probability, mu, sigma) 
  # lower bound should have tail_probability below it 
  lower_bound = normal_upper_bound( tail_probability, mu, sigma) 
  return lower_bound, upper_bound

mu_0, sigma_0 = normal_approximation_to_binomial( 1000, 0.5)
print(f"coin fliped 1,000 times: mu: {mu_0}, sigma: {sigma_0}\n")

def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    """
    How likely are we to see a value at least as extreme as x (in either
    direction) if our values are from a N(mu, sigma)?
    """
    if x >= mu:
        # x is greater than the mean, so the tail is everything greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is less than the mean, so the tail is everything less than x
        return 2 * normal_probability_below(x, mu, sigma)

two_sided_p_value(529.5, mu_0, sigma_0)   # 0.062

extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0    # Count # of heads
                    for _ in range(1000))                # in 1000 flips,
    if num_heads >= 530 or num_heads <= 470:             # and count how often
        extreme_value_count += 1                         # the # is 'extreme'

# p-value was 0.062 => ~62 extreme values out of 1000
#assert 59 < extreme_value_count < 65, f"{extreme_value_count}"

two_sided_p_value(531.5, mu_0, sigma_0)   # 0.0463


def estimated_parameters( N: int, n: int)-> Tuple[ float, float]: 
  p = n / N 
  sigma = math.sqrt( p * (1-p) / N) 
  return p, sigma

def a_b_test_statistic( N_A: int, n_A: int, N_B: int, n_B: int)-> float:
  p_A, sigma_A = estimated_parameters( N_A, n_A) 
  p_B, sigma_B = estimated_parameters( N_B, n_B) 
  return (p_B-p_A) / math.sqrt( sigma_A ** 2 + sigma_B ** 2)

z = a_b_test_statistic( 1000, 200, 1000, 180) #-1.14
tsv = two_sided_p_value( z) # 0.254
print(f"a_b test 200 180 probability: {tsv}\n")

z = a_b_test_statistic( 1000, 200, 1000, 150) #-2.94 
tsv = two_sided_p_value( z) # 0.003
print(f"a_b test 200 150 proabability: {tsv}\n")

coin fliped 1,000 times: mu: 500.0, sigma: 15.811388300841896

a_b test 200 180 probability: 0.254141976542236

a_b test 200 150 proabability: 0.003189699706216853

